In [ ]:
#注意！！在
###############################n
#中间的部分需要先改成自己的路径才能用！
###############################n

#依赖的库：pandas, requests, BeautifulSoup, openai（直接本地pip就行）
import pandas as pd
import requests
from bs4 import BeautifulSoup
import openai  #debug发现好像不能直接调默认的版本（经典版本过高，，），pip完之后跑起来要是有问题按照提示重新pip一个符合的版本就行了
import time

#设置OpenAI API密钥（这里如果没有的话也没必要买，我是在https://api.v36.cm/上薅的免费额度）
openai.api_key = "sk-TNiVDch5gjn5HV6W8332189f2fCf41849cA4B0880a7265A4"
openai.api_base = "https://api.vveai.com/v1"

#iGEM官网上的teams list，这里我保存成csv格式了，下载之后修改好路径就能直接用
#########################################################1
df = pd.read_csv("your_file_path\\teams.csv")
#########################################################1

# 筛选出2024年的wiki（这里改一改年份就可以调用前几年的）
filtered_urls = df[df['wiki'].astype(str).str.contains("2024")]['wiki'].tolist()
results = []
for url in filtered_urls:
    if url.endswith("/"):
        modified_url = url + "description"
    else:
        modified_url = url + "/description"
        
    print(f"modifying...：{modified_url}")
    try:
        response = requests.get(modified_url, timeout=10)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, "html.parser")
            for tag in soup(["script", "style"]):
                tag.extract()
            text = soup.get_text(separator=" ", strip=True)
            if len(text) > 7000:
                text = text[:7000]

            #这里构造调用ChatGPT的提示词（当然这个prompt可以按照自己想知道的内容改）
            prompt = (
                f"请分析以下从往年iGEM参赛队伍的wiki网页中提取的文本，"
                f"分析概述出该iGEM比赛参赛队伍的主要项目内容，包括：项目名称、问题背景、解决方案以及其它重要内容等，"
                f"要求结合文本内容并分点给出。以下是需要分析的文本：\n\n{text}"
            )

            #调用3.5接口（目前免费且不会卡bug或者直接宕机的我只找到3.5lol
            chat_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.61,
            )
            answer = chat_response['choices'][0]['message']['content']
            results.append({"url": modified_url, "content": answer})
        else:
            results.append({"url": modified_url, "content": f"failed, ：{response.status_code}"})
    except Exception as e:
        results.append({"url": modified_url, "content": f"error：{e}"})
    
    time.sleep(1.00)

result_df = pd.DataFrame(results)

#记得修改输出路径（不改也把your_file_path删掉）
#########################################################2
result_df.to_csv("your_file_path\\output.csv", index=False)
#########################################################2

print("输入csv中的wiki已全部解析完成，并已保存到output.csv。")